In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import pandas as pd
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt
sys.path.append('..')
from utils.data import Subject, Block

In [7]:
base_dir = '/Users/hugofluhr/data/LH_dev'
sub_ids = ['01', '02', '03', '05', '06', '09']

In [8]:
subjects = [Subject(base_dir, 'sub-'+sub_id, skip_imaging=True) for sub_id in sub_ids]

/Users/hugofluhr/phd_local/repositories/RewardPairsTask_Analysis/notebooks/../utils/data.py:162: UserWarning: Last 1 trial(s) of block had no response, filling with 0


In [9]:
sub = subjects[0]

In [10]:
sub.learning_phase[0].trials

,left_stim,right_stim,left_value,right_value,shift,action,rt,chosen_stim,reward,correct,t_first_stim,t_second_stim,t_action,t_purple_frame,t_points_feedback,t_iti_onset,t_trial_end
trial,,,,,,,,,,,,,,,,,
1,5,7,3,4,0,1.0,0.934432,5.0,3.0,0.0,0.009399,0.826556,1.760988,1.766016,2.268539,3.335600,10.010661
2,4,6,3,4,1,1.0,0.645392,4.0,3.0,0.0,10.010661,10.895611,11.541003,11.543884,12.045623,13.401687,20.403639
3,3,1,2,1,0,NaN,NaN,NaN,NaN,NaN,20.403639,21.369311,0.000000,0.000000,0.000000,23.875462,30.675264
4,6,8,4,5,0,1.0,0.368762,6.0,4.0,0.0,30.675264,31.478573,31.847335,31.849628,32.351760,33.984344,40.081664
5,2,4,2,3,0,1.0,0.537730,2.0,2.0,0.0,40.081664,40.994311,41.532041,41.534648,42.036649,43.500880,49.945219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,6,4,4,3,1,1.0,0.495369,6.0,4.0,1.0,919.311542,920.157687,920.653056,920.654386,921.155510,922.661213,928.780662
93,5,7,3,4,0,2.0,0.722157,7.0,4.0,1.0,928.780662,929.719888,930.442045,930.442927,930.944035,932.222907,938.515075
94,6,8,4,5,1,2.0,0.562463,8.0,5.0,1.0,938.515075,939.448659,940.011122,940.012432,940.513359,941.951858,949.332340


## Events in a trial:
- first_stim_only
- both_stim_presentation
- response (key press)
- purple frame displayed with response
- points displayed
- iti

In [11]:
sub.get_event_df('test')

,onset,duration,trial_type
0,0.002346,0.813894,first_stim_presentation
1,0.816239,0.550671,second_stim_presentation
2,1.366910,0.000000,response
3,1.370405,1.951056,purple_frame
4,3.321461,6.203388,iti
...,...,...,...
674,1360.803064,0.859366,first_stim_presentation
675,1361.662430,0.730383,second_stim_presentation
676,1362.392813,0.000000,response
677,1362.394750,1.770824,purple_frame
